# Use Langchain to implement RAG
First you implement the most basic RAG system possible using Langchain. When the defaults work, you make some adjustments to better control the output of the RAG system. 

## The basic RAG using defaults
You need an LLM and a Retriever. You can choose OpenAI and Weaviate or alternatives. Use the Langchain wrappers to initialise them.

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()
# TODO: Implement the retriever through the vector store.

import weaviate
from langchain.vectorstores import Weaviate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

weaviate_url = os.getenv('WEAVIATE_CLUSTER_URL')

auth_config = weaviate.auth.AuthApiKey(
    api_key=os.getenv('WEAVIATE_API_KEY'),
)

weaviate_client = weaviate.Client(
    url=weaviate_url,
    auth_client_secret=auth_config,
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv('OPEN_AI_API_KEY')
    }
)

llm = ChatOpenAI(openai_api_key=os.getenv('OPEN_AI_API_KEY'), model_name="gpt-4")

openai_embeddings = OpenAIEmbeddings(
    openai_api_key=os.getenv('OPEN_AI_API_KEY'),
    model="text-embedding-ada-002"
)

vector_store = Weaviate(client=weaviate_client,
                        embedding=openai_embeddings,
                        index_name="DevoxxFAQ",
                        text_key="text",
                        by_text=False)
             

In [ ]:
# TODO: Implement the chain using Chroma or Weaviate
from langchain.chains import RetrievalQA

question_answer_chain = RetrievalQA.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)


In [ ]:
query="How many proposals are received, and how many proposals can be accepted?"
print(question_answer_chain.run(query))

In [ ]:
# TODO: Experiment with more questions

## Replacing the defaults
You have used most of the defaults. Still you can configure the different components using the same structure.

This one can be a bit tricky. You need to check the documents that were used to generate the answer. There is an option to also return the source documents. On which component would you add that option? Make the change so that the following block runs without errors.

In [ ]:
# TODO change the previous code blocks in such a way that the next code blocks runs without errors and shows the result as well as the source documents.

query="How many proposals are received, and how many proposals can be accepted?"
response = question_answer_chain({"query": query})

print(response["result"])
print("--------------")
print("The source documents")
for doc in response["source_documents"]: 
    print(doc.page_content)
    print("-----")


Next you want to be friendly to non-native english speakers and make it possible to return the answers in another language. Change the prompt in such a way that the result will be printed in Dutch.

In [ ]:
# TODO Adjust the prompt in such a way that the result is in another language.

from langchain import PromptTemplate, OpenAI

prompt_template = """
    You are a system that writes an answer to the provided question using the provided context. Answer in Dutch.
    
    {context}

    Question: {question}:"""

custom_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

question_answer_chain = RetrievalQA.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    prompt=custom_prompt,
    return_source_documents=True
)


In [ ]:
query = "How many proposals are received, and how many proposals can be accepted?"
response = question_answer_chain({"query": query})

print(response["result"])
print("--------------")
print("The source documents")
for doc in response["source_documents"]:
    print(doc.page_content)
    print("-----")
